1.
(15 Pts.) Construct an array of 1 million numbers in the range between (-10000.00 to 10000.00).
•
Write a code that gives the prime numbers in form of a list without using any library that directly gives the prime numbers as an output.
•
Write a code that sums up the square of all the negative numbers and cubes of all the positive numbers. Can this value ever be negative?
•
Write an algorithm that sorts the number in an arrangement where the first number is the largest, second number is Smallest, third number is second largest, fourth number is second smallest so on.
i.
For example, if input is: [-5, -3.2, 0, 1, 3, -6,10]
ii.
then the output should be: [10, -6, 3, -5, 1 -3.2, 0]


In [2]:
import numpy as np

# array construction  
array = np.random.uniform(-10000.00, 10000.00, 1000000)


In [3]:
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(abs(num)**0.5) + 1):
        if num % i == 0:
            return False
    return True

# prime numbers Filter 
prime_numbers = [int(num) for num in array if is_prime(int(num))]

# Display of first 10 numbers found 
print(prime_numbers[:10])


[443, 7577, 4099, 8803, 3011, 5309, 337, 5351, 9311, 47]


In [4]:
# Sum squares of negative numbers and cubes of positive numbers
sum_squares_neg = np.sum([num**2 for num in array if num < 0])
sum_cubes_pos = np.sum([num**3 for num in array if num > 0])

result_sum = sum_squares_neg + sum_cubes_pos

#result
print(f"Sum of squares of negatives and cubes of positives: {result_sum}")


Sum of squares of negatives and cubes of positives: 1.2503237089898534e+17


In [5]:
import time

def custom_sort(arr):
    arr_sorted = sorted(arr)
    result = []
    i, j = 0, len(arr_sorted) - 1
    while i <= j:
        if i != j:
            result.append(arr_sorted[j])
            result.append(arr_sorted[i])
        else:
            result.append(arr_sorted[j])
        i += 1
        j -= 1
    return result

# Time the sorting process
start_time = time.time()

# Sort the array
sorted_array = custom_sort(array)

execution_time = time.time() - start_time

# first 10 elements of the sorted array and the execution time
print(f"Sorted array (first 10 elements): {sorted_array[:10]}")
print(f"Execution time: {execution_time} seconds")


Sorted array (first 10 elements): [9999.943769435831, -9999.95873479853, 9999.943337811023, -9999.950809588165, 9999.941168299796, -9999.948695162515, 9999.94077643859, -9999.942763183843, 9999.921162613682, -9999.898827449631]
Execution time: 1.4552934169769287 seconds


Load the following three Yelp dataset (mentioned below). You can read more about the Yelp review data here. These datasets are relatively large so avoid directly loading it into your memory/data frame. The objective of this task is to assess your ability to handle large data efficiently even under resource constraint.
i.
yelp_academic_dataset_review
ii.
yelp_academic_dataset_business
iii.
yelp_academic_dataset_user
•
Perform the following task once all your 3 datasets are in place:

Loading the Yelp Datasets

In [55]:
import pandas as pd

chunk_memory = []
def load_and_analyze (file_name):
    for chunk in pd.read_json(file_name,lines=True, chunksize=10000):
#         print(pd.read_json(file_name).column.dtype)
        chunk_memory.append(chunk.memory_usage(deep=True).sum())
        
    total_memory = sum(chunk_memory)
    print(total_memory/1024**2)
        
review_chunks = load_and_analyze('yelp_academic_dataset_review.json')
business_chunks = load_and_analyze('yelp_academic_dataset_business.json')
user_chunks = load_and_analyze('yelp_academic_dataset_user.json')


6023.610883712769
6177.040030479431
9474.322436332703


In [44]:
# def optimize_data_types(path):
import pandas as pd
data = pd.read_json('yelp_academic_dataset_review.json',line=True)
for col in df:
#     select_dtype(include =['object']).columns:
    print(col)
print(df.dtypes)
#         df[col] = df[col].astype('category')
        
# review_chunks = optimize_data_types('yelp_academic_dataset_review.json')
# business_chunks = optimize_data_types(pd.read_json('yelp_academic_dataset_business.json'))
# user_chunks = optimize_data_types(pd.read_json('yelp_academic_dataset_user.json'))


TypeError: read_json() got an unexpected keyword argument 'line'

In [50]:
import pandas as pd
import numpy as np

def load_data(file_path):
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith(('.xls', '.xlsx')):
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

def process_tax_audit_data(sample_data_path, master_data_path, output_path_1, output_path_2):
    
    sample_data = load_data(sample_data_path)
    master_data = load_data(master_data_path)
    
    # Propagate Date, Vch Type, Vch No down the group
    sample_data['Date'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Date'].transform('first')
    sample_data['Vch Type'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Vch Type'].transform('first')
    sample_data['Vch No.'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Vch No.'].transform('first')
    
    sample_data['Particulars'] = sample_data['Particulars'].astype(str)
    
    # narration rows and bring narration to a new column
    sample_data['Narration'] = np.where(sample_data['Particulars'].str.contains('Narration',na=False), sample_data['Particulars'], np.nan)
    sample_data['Narration'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Narration'].ffill().bfill()
    sample_data = sample_data[~sample_data['Particulars'].str.contains('Narration', na=False)]
    
    # Debit and Credit amounts
    sample_data['Debit Amount'] = sample_data['Debit Amount'].abs()
    sample_data['Credit Amount'] = -sample_data['Credit Amount'].abs()
    
    # V-Lookup from Master file
    sample_data = pd.merge(sample_data, master_data[['Particulars', 'Header']], on='Particulars', how='left')
    
    # first output (Processed Data)
    sample_data.to_csv(output_path_1, index=False)
    
    # data for Output 2
    fixed_particular = sample_data['Particulars'].iloc[0]
    reshaped_data = sample_data.pivot(index=['Date', 'Vch Type', 'Vch No.'], columns='Particulars', values='Amount').reset_index()
    reshaped_data.insert(0, 'Fixed Particular', fixed_particular)
    
    # second output (Reshaped Data)
    reshaped_data.to_csv(output_path_2, index=False)
    print(f"Processed data saved to {output_path_1} and reshaped data saved to {output_path_2}")

# Example usage
sample_data_path = 'Sample Data.csv'  # Replace with actual file path
master_data_path = 'Master.csv'       # Replace with actual file path
output_path_1 = 'Processed_Data.csv'
output_path_2 = 'Reshaped_Data.csv'

process_tax_audit_data(sample_data_path, master_data_path, output_path_1, output_path_2)


ValueError: Index contains duplicate entries, cannot reshape

In [56]:
import pandas as pd
 
business = pd.read_json('yelp_academic_dataset_review.json')
review = pd.read_json('yelp_academic_dataset_business.json')
user = pd.read_json('yelp_academic_dataset_user.json')

def missing_data_info(df):
    missing_data = df.isnull().sum()  # Count missing values
    total_data = len(df)  # Total rows
    missing_percentage = (missing_data / total_data) * 100  # Percentage of missing values
    
    return pd.DataFrame({
        'Missing Values': missing_data,
        'Percentage': missing_percentage
    })

# missing data for each dataset
business_missing_data = missing_data_info(business)
review_missing_data = missing_data_info(review)
user_missing_data = missing_data_info(user)

# results
print("Business Dataset Missing Data:")
print(business_missing_data)

print("\nReview Dataset Missing Data:")
print(review_missing_data)

print("\nUser Dataset Missing Data:")
print(user_missing_data)

MemoryError: 

You have been provided with “Sample Data” and “Master” from a client that needs to be audited for tax. This data is coming from a system in forms for report. The client is requesting you to automate the process. You have been provided with the following requirements from a Chartered Accountant who does not have exposure to Python, SQL, or coding in general:

In [52]:
import pandas as pd
import numpy as np

def load_data(file_path):
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith(('.xls', '.xlsx')):
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

def process_tax_audit_data(sample_data_path, master_data_path, output_path_1, output_path_2):
 
    sample_data = load_data(sample_data_path)
    master_data = load_data(master_data_path)
    
    # Date, Vch Type, Vch No down the group
    sample_data['Date'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Date'].transform('first')
    sample_data['Vch Type'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Vch Type'].transform('first')
    sample_data['Vch No.'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Vch No.'].transform('first')
    
     sample_data['Particulars'] = sample_data['Particulars'].astype(str)
    
    # rows and bring narration to a new column
    sample_data['Narration'] = np.where(sample_data['Particulars'].str.contains('Narration', na=False), sample_data['Particulars'], np.nan)
    sample_data['Narration'] = sample_data.groupby(['Vch Type', 'Vch No.'])['Narration'].ffill().bfill()
    
    # rows that are identified as narration
    sample_data = sample_data[~sample_data['Particulars'].str.contains('Narration', na=False)]
    
    # Debit and Credit amounts
    sample_data['Debit Amount'] = sample_data['Debit Amount'].abs()
    sample_data['Credit Amount'] = -sample_data['Credit Amount'].abs()
    
    # V-Lookup from Master file
    sample_data = pd.merge(sample_data, master_data[['Particulars', 'Header']], on='Particulars', how='left')
    
    # first output (Processed Data)
    sample_data.to_csv(output_path_1, index=False)
    
    # data for Output 2 using pivot_table to handle duplicates
    reshaped_data = sample_data.pivot_table(
        index=['Date', 'Vch Type', 'Vch No.'],
        columns='Particulars',
        values='Amount',
        aggfunc='sum'  # You can change this to other functions depending on your needs
    ).reset_index()
    
    # Concatenate fixed particular
    fixed_particular = sample_data['Particulars'].iloc[0]
    reshaped_data.insert(0, 'Fixed Particular', fixed_particular)
    
    # second output (Reshaped Data)
    reshaped_data.to_csv(output_path_2, index=False)
    print(f"Processed data saved to {output_path_1} and reshaped data saved to {output_path_2}")

# Example usage
sample_data_path = 'Sample Data.csv'  # Replace with actual file path
master_data_path = 'Master.csv'       # Replace with actual file path
output_path_1 = 'Processed_Data.csv'
output_path_2 = 'Reshaped_Data.csv'

process_tax_audit_data(sample_data_path, master_data_path, output_path_1, output_path_2)


Processed data saved to Processed_Data.csv and reshaped data saved to Reshaped_Data.csv
